In [ ]:
import requests

app_id = 'd1f4cfd5-2a86-4c76-8593-5f999bf1a02d' 
api_key = 'MLXMoaTgjEozDqGUPJbWBsYQoKCwZ06n'

In [ ]:
import requests
import pyjq
import pandas as pd
import datetime

def get_text():
    app_id = 'd1f4cfd5-2a86-4c76-8593-5f999bf1a02d' 
    api_key = 'MLXMoaTgjEozDqGUPJbWBsYQoKCwZ06n'
    
    data_dict = []
    for i in range(1,11):
        api_key = 'MLXMoaTgjEozDqGUPJbWBsYQoKCwZ06n'
        url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=s&n500&page='+str(i)+'&api-key=' +api_key
        r = requests.get(url, timeout=10)
        print('requesting page ', i)
        data = r.json()
        data_dict.append(data)
    
    data_list = []
    for i in data_dict:
        jq_query = f'.response .docs [] | {{url: .web_url, snippet: .snippet, date: .pub_date}}'
        output = pyjq.all(jq_query, i)
        data_list.extend(output)
        print(data_list)
        print(len(data_list))

    df = pd.DataFrame(data_list)
    df.drop_duplicates(keep=False, inplace=True)
    df['date'] = df['date'].map(lambda i: str(i)[:10])
    df['date'] = df['date'].map(lambda i: datetime.datetime.strptime(i, '%Y-%m-%d'))
    df.sort_values('date', inplace=True)
    return df


In [ ]:
data_dict = []
for i in range(1,11):
    api_key = 'MLXMoaTgjEozDqGUPJbWBsYQoKCwZ06n'
    url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?q=s&n500&page='+str(i)+'&api-key=' +api_key
    r = requests.get(url, timeout=10)
    print('requesting page ', i)
    data = r.json()
    data_dict.append(data)
    print(data_dict)

In [ ]:
data_list = []
for i in data_dict:
    jq_query = f'.response .docs [] | {{url: .web_url, snippet: .snippet, date: .pub_date}}'
    output = pyjq.all(jq_query, i)
    data_list.extend(output)
    print(data_list)
    print(len(data_list))

In [ ]:
df = pd.DataFrame(data_list)
df.drop_duplicates(keep=False, inplace=True)
df['date'] = df['date'].map(lambda i: str(i)[:10])
df['date'] = df['date'].map(lambda i: datetime.datetime.strptime(i, '%Y-%m-%d'))
df.sort_values('date', inplace=True)

In [ ]:
df

In [ ]:
import json

article_dict = df.to_dict(orient='records')
article_dict

# Webscraping each article with URL from API

In [ ]:
from bs4 import BeautifulSoup as BS


def get_nyt_text(url):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    page = requests.get(url, headers=headers,timeout=5)
    page.status_code
    
    soup = BS(page.content, 'html.parser')
    content = soup.findAll('p', class_ = 'css-18icg9x evys1bk0')
    
    nyt = ''
    for index in range(len(content)):
        nyt += content[index].get_text()
        
    return nyt

In [ ]:
for i in article_dict:
    i.update({'article':get_nyt_text(i['url'])})
article_dict

In [ ]:
len(article_dict)

# Using Vader to evaluate sentiment of each article

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
si = SentimentIntensityAnalyzer()

for i in article_dict:
    i.update({'sentiment':si.polarity_scores(i['article'])})
article_dict

In [ ]:
for i in article_dict:
    i.update({'compound': i['sentiment']['compound']})
article_dict

In [ ]:
df_sentiment = pd.DataFrame(article_dict)
df_sentiment

In [ ]:
df_sentiment.drop(columns=['article', 'sentiment', 'url', 'snippet'], inplace=True)
df_sentiment

In [ ]:
daily_sentiment = df_sentiment.groupby('date').mean()
daily_sentiment

In [ ]:
TSLA = pd.read_csv('TSLA.csv')
TSLA

In [ ]:
TSLA.drop(columns=['High', 'Low', 'Adj Close', 'Volume'], inplace=True)
TSLA['Date'] = TSLA['Date'].map(lambda i: datetime.datetime.strptime(i, '%Y-%m-%d'))

In [ ]:
TSLA['Daily_Change'] = TSLA['Close'] / TSLA['Open'] -1

%matplotlib inline 
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import statsmodels.api as sm

TSLA.plot(figsize=(12,8), x='Date', y='Daily_Change', title= 's&n500 Daily Performance', fontsize=14)

In [ ]:
TSLA.plot(figsize=(12,8), x='Date', y='Close', title= 's&n500 Daily Close', fontsize=14)

In [ ]:
TSLA.set_index('Date', inplace=True)
TSLA.drop(columns=['Open', 'Daily_Change'], inplace=True)
TSLA

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(TSLA, freq=12)
fig = plt.figure()
fig = decomposition.plot()
fig.set_size_inches(15, 8)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(TSLA.Close.iloc[1:], lags=18, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(TSLA.Close.iloc[1:], lags=18, ax=ax2)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def test_stationarity(timeseries, window):
    
    rolmean = timeseries.rolling(window=window).mean()
    rolstd = timeseries.rolling(window=window).std()

    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries.iloc[window:], color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

test_stationarity(TSLA.Close, 12)

# Stationarize the data

In [ ]:
import numpy as np

TSLA['natural_log'] = TSLA.Close.apply(lambda x: np.log(x))  
test_stationarity(TSLA.natural_log, 12)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(TSLA.natural_log.iloc[1:], lags=18, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(TSLA.natural_log.iloc[1:], lags=18, ax=ax2)

In [ ]:

TSLA['first_difference'] = TSLA.Close - TSLA.Close.shift(1)  
test_stationarity(TSLA.first_difference.dropna(inplace=False),12)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(TSLA.first_difference.iloc[1:], lags=18, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(TSLA.first_difference.iloc[1:], lags=18, ax=ax2)

In [ ]:
TSLA['seasonal_difference'] = TSLA.Close - TSLA.Close.shift(12)  
test_stationarity(TSLA.seasonal_difference.dropna(inplace=False), 12)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(TSLA.seasonal_difference.iloc[13:], lags=24, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(TSLA.seasonal_difference.iloc[13:], lags=24, ax=ax2)

In [ ]:
TSLA['seasonal_first_difference'] = TSLA.first_difference - TSLA.first_difference.shift(12)  
test_stationarity(TSLA.seasonal_first_difference.dropna(inplace=False), 12)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(TSLA.seasonal_first_difference.iloc[13:], lags=24, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(TSLA.seasonal_first_difference.iloc[13:], lags=24, ax=ax2)

In [ ]:
TSLA['log_first_difference'] = TSLA.natural_log - TSLA.natural_log.shift(1)  
TSLA['log_seasonal_first_difference'] = TSLA.log_first_difference - TSLA.log_first_difference.shift(12)  

test_stationarity(TSLA.log_seasonal_first_difference.dropna(inplace=False), 12)


In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(TSLA.log_seasonal_first_difference.iloc[13:], lags=24, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(TSLA.log_seasonal_first_difference.iloc[13:], lags=24, ax=ax2)

# Examining data with sentiment compound

In [ ]:
TSLA_FD = pd.DataFrame(TSLA.first_difference)
TSLA_FD

In [ ]:
TSLA_FD_COMP = daily_sentiment.merge(TSLA_FD, left_index=True, right_index=True)
TSLA_FD_COMP

In [ ]:
fig, ax1 = plt.subplots(1,1)

TSLA_FD_COMP.plot(ax=ax1, y='first_difference', use_index=True, figsize=(10,7), legend=True, c='r')

ax2 = ax1.twinx()

TSLA_FD_COMP.plot(ax=ax2, y='compound', use_index=True, figsize=(10,7), legend=True)

ax1.set_ylabel("first_difference", fontsize=10)
ax1.set_xlabel("compound", fontsize=10)


plt.show()


In [ ]:
TSLA_FD_COMP.plot.scatter(x='compound', y='first_difference')

# Examining data with sentiment elements

In [ ]:
article_dict

In [ ]:
for i in article_dict:
    i.update({'positive': i['sentiment']['pos']})
    i.update({'negative': i['sentiment']['neg']})
    i.update({'neutral': i['sentiment']['neu']})
article_dict

In [ ]:
sentiments = pd.DataFrame(article_dict)
sentiments

In [ ]:
sentiments.drop(columns=['article', 'compound', 'sentiment', 'snippet', 'url'], inplace=True)

In [ ]:
sentiments = sentiments.groupby('date').mean()
sentiments

In [ ]:
TSLA_FD_SENT = sentiments.merge(TSLA_FD, left_index=True, right_index=True)
TSLA_FD_SENT

In [ ]:
fig, ax1 = plt.subplots(1,1)

TSLA_FD_SENT.plot(ax=ax1, y='first_difference', use_index=True, figsize=(10,7), legend=True, c='r')

ax2 = ax1.twinx()

TSLA_FD_SENT.plot(ax=ax2, y='negative', use_index=True, figsize=(10,7), legend=True)
TSLA_FD_SENT.plot(ax=ax2, y='neutral', use_index=True, figsize=(10,7), legend=True)
TSLA_FD_SENT.plot(ax=ax2, y='positive', use_index=True, figsize=(10,7), legend=True)

plt.show()

In [ ]:
TSLA_FD_SENT.plot.scatter(x='positive', y='first_difference')
TSLA_FD_SENT.plot.scatter(x='negative', y='first_difference')
TSLA_FD_SENT.plot.scatter(x='neutral', y='first_difference')

# Converting target variable into binary data - Vader for snippets

In [ ]:
import json

with open('nyt_api_2010_2019.json') as f:
    d = json.load(f)
    print(d)

In [ ]:
sentiment = pd.read_pickle("sentiment_counts.pkl")

sentiment.index = pd.DatetimeIndex(sentiment.index)
sentiment

In [ ]:
TSLA_DF = pd.read_csv('TSLA 2010.csv')
TSLA_DF['First_Diff'] = TSLA_DF.Close - TSLA_DF.Close.shift(1)
TSLA_DF['Date'] = pd.to_datetime(TSLA_DF.Date)

In [ ]:
TSLA_sentiment = sentiment.merge(TSLA_DF, left_index=True, right_on='Date')

In [ ]:
TSLA_sentiment['Movement'] = np.where(TSLA_sentiment['First_Diff'] > 0, 1, 0)

In [ ]:
TSLA_sentiment_Cleaned = TSLA_sentiment[TSLA_sentiment['VADER snippet compound'] != 0]
TSLA_sentiment_Cleaned.set_index('Date', inplace=True)
TSLA_sentiment_Cleaned

In [ ]:
Nasdaq = pd.read_csv('Nasdaq2010-2019.csv')
Nasdaq['Nasdaq_First_Diff'] = Nasdaq.Close - Nasdaq.Close.shift(1)
Nasdaq['Nasdaq_Movement'] = np.where(Nasdaq['Nasdaq_First_Diff'] > 0, 1, 0)
Nasdaq['Date'] = pd.to_datetime(Nasdaq.Date)
Nasdaq

In [ ]:
Combined = TSLA_sentiment_Cleaned.merge(Nasdaq, left_index=True, right_on='Date')
Combined

In [ ]:
Cleaned = Combined[Combined['VADER snippet compound'] != 0]
Cleaned.set_index('Date', inplace=True)
Cleaned

In [ ]:
Cleaned.columns

### Baseline Log Model

In [ ]:
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures

from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

In [ ]:
X = Cleaned[['VADER snippet compound', 'VADER snippet neg', 'VADER snippet neu', 'VADER snippet pos', 'article count', 'Volume_x', 'Nasdaq_Movement']]
y = Cleaned['Movement']

columns = X.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train =pd.DataFrame(data=scaler.transform(X_train), columns=columns)
X_test =pd.DataFrame(data=scaler.transform(X_test), columns=columns)

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(fit_intercept = False, C = 1e12)
logreg.fit(X_train, y_train)
dictionary = dict(zip(list(X_train.columns), list(logreg.coef_[0])))
dictionary

In [ ]:
y_pred_class = logreg.predict(X_test)

from sklearn import metrics
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_class)
classes = ['Moved Up', "Moved Down"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
import itertools
plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)

### Naives Baye model

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
y_pred.shape

In [ ]:
clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
classes = ['Moved Up', "Moved Down"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
import itertools
plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)

In [ ]:
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred))
print('Test F1 score: ', metrics.f1_score(y_test, y_pred))

# Using Textblob for article

In [ ]:
from textblob import TextBlob

sentiment_article = pd.read_pickle("sentiment_counts.pkl")

sentiment_article.index = pd.DatetimeIndex(sentiment_article.index)
sentiment_article

In [ ]:
Combined_TB = sentiment_article.merge(Combined, left_index=True, right_on='Date')
Combined_TB

In [ ]:
Cleaned_TB = Combined_TB[Combined_TB['TextBlob article polarity_x'] != 0]
Cleaned_TB.set_index('Date', inplace=True)
Cleaned_TB.columns

### Log Model

In [ ]:
X = Cleaned_TB[['TextBlob article polarity_x', 'TextBlob article subjectivity_x','article count_y',  'Volume_x', 'Nasdaq_Movement']]
y = Cleaned_TB['Movement']

columns = X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)

scaler = StandardScaler()
scaler.fit(X_train)
X_train =pd.DataFrame(data=scaler.transform(X_train), columns=columns)
X_test =pd.DataFrame(data=scaler.transform(X_test), columns=columns)

logreg = LogisticRegression(fit_intercept = False, C = 1e12)
logreg.fit(X_train, y_train)
dictionary = dict(zip(list(X_train.columns), list(logreg.coef_[0])))
dictionary

In [ ]:
y_pred_class = logreg.predict(X_test)

print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred_class)
classes = ['Moved Up', "Moved Down"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
import itertools
plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)

### NB Model

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.predict_proba(X_test)

In [ ]:
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred))
print('Test F1 score: ', metrics.f1_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
classes = ['Moved Up', "Moved Down"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
import itertools
plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)

# Using Vader and TextBlob as features

In [ ]:
X = Cleaned_TB[['VADER snippet compound_x', 'VADER snippet neg_x','VADER snippet neu_x', 'VADER snippet pos_x', 'article count_x', 'TextBlob article polarity_x', 'TextBlob article subjectivity_x', 'Volume_x', 'Nasdaq_Movement']]
y = Cleaned_TB['Movement']

columns = X.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)

scaler = StandardScaler()
scaler.fit(X_train)
X_train =pd.DataFrame(data=scaler.transform(X_train), columns=columns)
X_test =pd.DataFrame(data=scaler.transform(X_test), columns=columns)

logreg = LogisticRegression(fit_intercept = False, C = 1e12)
logreg.fit(X_train, y_train)
dictionary = dict(zip(list(X_train.columns), list(logreg.coef_[0])))
dictionary

In [ ]:
y_pred_class = logreg.predict(X_test)

print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_class))
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_class))

In [ ]:
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.predict_proba(X_test)


In [ ]:
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred))
print('Test F1 score: ', metrics.f1_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
classes = ['Moved Up', "Moved Down"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion Matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
import itertools
plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues)